In [5]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm

In [2]:
file = open('CA 2019 Permits.txt','r')

text_data = [x.split('\t') for x in file.read().split('\n') if x!='Note: i = Imputed Value  ']

In [4]:
list_of_permit_authorities = []
for n,line in enumerate(text_data):
    if n%26==0:
        list_of_permit_authorities.append(line[0] + f'|{text_data[n+1][0]}')

single_family = []
for n,line in enumerate(text_data):
    if (n-19)%26==0:
        single_family.append(line)

multi_family = []
for n,line in enumerate(text_data):
    if (n-20)%26==0:
        multi_family.append(line)

two_unit = []
for n,line in enumerate(text_data):
    if (n-21)%26==0:
        two_unit.append(line)
        
three_unit = []
for n,line in enumerate(text_data):
    if (n-22)%26==0:
        three_unit.append(line)
        
five_unit = []
for n,line in enumerate(text_data):
    if (n-23)%26==0:
        five_unit.append(line)
        
        
dict_of_permit_authorities = {xs:{} for xs in list_of_permit_authorities}

In [6]:
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','Total']
for n,x in enumerate(single_family):
    key = list(dict_of_permit_authorities.keys())[n]
    
    permit_months = {month:x[1:][n] for n,month in enumerate(months)}
    dict_of_permit_authorities[key][x[0]] = permit_months
    
for n,x in enumerate(multi_family):
    key = list(dict_of_permit_authorities.keys())[n]
    
    permit_months = {month:x[1:][n] for n,month in enumerate(months)}
    dict_of_permit_authorities[key][x[0]] = permit_months 
    
for n,x in enumerate(two_unit):
    key = list(dict_of_permit_authorities.keys())[n]
    
    permit_months = {month:x[1:][n] for n,month in enumerate(months)}
    dict_of_permit_authorities[key][x[0]] = permit_months
    
for n,x in enumerate(three_unit):
    key = list(dict_of_permit_authorities.keys())[n]
    
    permit_months = {month:x[1:][n] for n,month in enumerate(months)}
    dict_of_permit_authorities[key][x[0]] = permit_months
    
for n,x in enumerate(five_unit):
    key = list(dict_of_permit_authorities.keys())[n]
    
    permit_months = {month:x[1:][n] for n,month in enumerate(months)}
    dict_of_permit_authorities[key][x[0]] = permit_months

In [23]:
ret_df = pd.DataFrame(columns = ['Permit Authority', 'County', 'Month', 'Year', 'Permit Type', '# of Permits'])

permit_types = ['Units in Single-Family Structures',
                'Units in All Multi-Family Structures',
                'Units in 2-unit Multi-Family Structures',
                'Units in 3- and 4-unit Multi-Family Structures',
                'Units in 5+ Unit Multi-Family Structures',
               ]

for authority in tqdm(dict_of_permit_authorities):
    permit_authority = authority.split('|')[0]
    county = authority.split('|')[1]
    for permit_type in permit_types:
        concat_df = pd.DataFrame(index = range(13), columns = ['Permit Authority', 'County', 'Month', 'Year', 'Permit Type', '# of Permits'])

        for n,month in enumerate(months):
            permits = dict_of_permit_authorities[authority][permit_type][month]
            concat_df.loc[n] = [permit_authority, county, month, 2019, permit_type, permits]
            
        ret_df = pd.concat([ret_df, concat_df])

ret_df.reset_index(drop=True, inplace=True)
            

100%|████████████████████████████████████████████████████████████████████████████████| 470/470 [00:05<00:00, 83.12it/s]


In [24]:
#ret_df.to_csv('2019_test.csv',index=False)

# Check cities with Charters

In [23]:
permits_by_authority = pd.read_csv('permits_by_permitting_authority.csv')

file = open('city_charters/California Charter Cities.txt','r')
charter_cities = file.read().split('\n')
file.close()

In [24]:
charter_cities_df = pd.DataFrame([(city.upper() + ', CA','charter') for city in charter_cities], 
                                 columns=["Permit Authority", "city form"]
                                )

In [25]:
xx = permits_by_authority\
    .merge(charter_cities_df, on='Permit Authority', how='left')\

xx['city form'] = xx['city form'].fillna('general law')

xx['city form'] = [form if 'UNINCORPORATED' not in authority else 'county' for form,authority in zip(xx['city form'],xx['Permit Authority'])]

In [26]:
(set([city.upper() + ', CA' for city in charter_cities])-set(xx[xx['city form']=='charter']['Permit Authority']))

{'EUREKA, CA', 'FORTUNA, CA', 'GRASS VALLEY, CA', 'TRUCKEE, CA', 'VENTURA, CA'}

In [27]:
xx.to_csv('permits_by_permitting_authority_city_form.csv', index=False)

# Check Counties with Charters

In [31]:
county_types = pd.read_excel('city_charters/county government type (wikipedia).xlsx') \
                .dropna() \
                .rename(columns={'county':'County', 'gov type':'County gov type', 'population':'County population'}) \
                .reset_index(drop=True) \
                [['County', 'County gov type', 'County population']]

permits_by_authority = pd.read_csv('permits_by_permitting_authority.csv')

In [32]:
for row in range(len(county_types)):
    if county_types.County[row]=='San Francisco':
        county_types.County[row]='San Francisco County'

In [33]:
permits_by_authority = permits_by_authority.merge(county_types, on='County', how='left')

,Permit Authority,County,Month,Year,Permit Type,# of Permits,notes,city form,County gov type,County population
0,"ALAMEDA COUNTY UNINCORPORATED AREA, CA",Alameda County,Jan,2014,Units in Single-Family Structures,1,NaN,county,Charter,1671329.0
1,"ALAMEDA COUNTY UNINCORPORATED AREA, CA",Alameda County,Feb,2014,Units in Single-Family Structures,1,NaN,county,Charter,1671329.0
2,"ALAMEDA COUNTY UNINCORPORATED AREA, CA",Alameda County,Mar,2014,Units in Single-Family Structures,1,NaN,county,Charter,1671329.0
3,"ALAMEDA COUNTY UNINCORPORATED AREA, CA",Alameda County,Apr,2014,Units in Single-Family Structures,3,NaN,county,Charter,1671329.0
4,"ALAMEDA COUNTY UNINCORPORATED AREA, CA",Alameda County,May,2014,Units in Single-Family Structures,3,NaN,county,Charter,1671329.0
...,...,...,...,...,...,...,...,...,...,...
183425,"WHEATLAND, CA",Yuba County,Sep,2019,Units in 5+ Unit Multi-Family Structures,0,NaN,general law,General Law,78668.0
183426,"WHEATLAND, CA",Yuba County,Oct,2019,Units in 5+ Unit Multi-Family Structures,0,NaN,general law,General Law,78668.0
183427,"WHEATLAND, CA",Yuba County,Nov,2019,Units in 5+ Unit Multi-Family Structures,0,NaN,general law,General Law,78668.0
183428,"WHEATLAND, CA",Yuba County,Dec,2019,Units in 5+ Unit Multi-Family Structures,0,NaN,general law,General Law,78668.0
